### Ejercicio
- Como simulacion de datos tomaremos los locales y licencias comerciales y crearemos datos sinteticos para poder visualizar la efectividad o la dinamica del RAG para puesta en produccion

In [1]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
import utm
import sqlite3
import requests
from functions.cleaning_extracting import (list_of_columns_function,
                                                    visualization_of_nan,
                                                    cleaner,
                                                    hourglass,
                                                    features_ing,
                                                    )

pd.reset_option("max_columns")
pd.set_option("max_colwidth",None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [4]:
df_actividad = pd.read_csv("../notebooks/data/actividadeconomica202401.csv",sep=";",low_memory=False,encoding="UTF-8")
#CSV de actividades comerciales en Madrid (extraido de una web publica-https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default-)

In [5]:
df_actividad= df_actividad.drop(columns=["fx_datos_ini","fx_datos_fin","id_local_agrupado","desc_tipo_agrup",\
                            "id_tipo_agrup","nombre_agrupacion","id_agrupacion","coordenada_y_agrupacion",\
                            "coordenada_x_agrupacion","cal_acceso","num_acceso","nom_acceso","id_clase_ndp_acceso",\
                            "id_ndp_acceso","id_vial_acceso","secuencial_local_PC","cal_edificio","num_edificio",\
                            "nom_edificio","id_ndp_edificio","desc_vial_edificio","clase_vial_edificio",\
                            "id_vial_edificio","desc_tipo_acceso_local","id_tipo_acceso_local","id_seccion_censal_local"])

In [8]:


"""
Eliminando features que no son necesarias para analisis
"""

df_licencias= pd.read_csv("../notebooks/data/licencias202408.csv",sep=";",encoding="UTF-8")#CSV de licencias en Madrid (extraido de una web publica-https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default-)
df_licencias.drop_duplicates(subset=['id_local'], keep='first', inplace=True)#Hay muchos locales que tienen mas de 1 licencia, un buen ejemplo son centros comerciales que tienen licencia de venta de alcohol y/o carniceria.
df_licencias= df_licencias[["id_local","ref_licencia","id_tipo_licencia","desc_tipo_licencia",\
                            "id_tipo_situacion_licencia","desc_tipo_situacion_licencia","Fecha_Dec_Lic"]]

"""
Extrayendo solo los features necesarios
"""

df_locales= pd.read_csv("../notebooks/data/locales202408.csv",sep=";",encoding="UTF-8")#CSV de locales comerciales en Madrid -https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default
df_locales= df_locales[["id_local","desc_situacion_local","id_situacion_local","coordenada_x_local","coordenada_y_local","cod_barrio_local"]]

#seleccion de features para analisis y trabajo

df_terrazas= pd.read_csv("../notebooks/data/terrazas202408.csv",sep=";",encoding="UTF-8")#CSV de terrazas en Madrid -https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default
df_terrazas= df_terrazas[["id_terraza","id_local","id_periodo_terraza","desc_periodo_terraza",\
                        "id_situacion_terraza","desc_situacion_terraza","Superficie_ES",\
                         "desc_ubicacion_terraza","hora_ini_LJ_es","hora_fin_LJ_es","mesas_es","sillas_es"]]

In [10]:
merged_df= pd.merge(df_locales,df_licencias, how= "outer")
merged_df_2= pd.merge(merged_df,df_terrazas,how="outer",on="id_local")
merged_df_3= pd.merge(merged_df_2,df_actividad,how="outer",on="id_local")#merged final con todos los datasets y sus features seleccionados
df_cleaned= merged_df_3.loc[merged_df_3['id_division'] == "56"]#seleccion de locales por id_division=56 (restaurantes o establecimientos de alimentos y bebidas)
df_cleaned= df_cleaned.reset_index()

In [11]:
#creando una mascara para salir de los negocios que no son propiamente locales de hosteleria
mask =(df_cleaned["desc_epigrafe"] == "SERVICIOS DE COMEDOR EN CENTROS PARA MAYORES")\
    | (df_cleaned["desc_epigrafe"] == "SERVICIOS DE COMEDOR EN CENTROS EDUCATIVOS Y CENTROS DE CUIDADO INFANTIL")\
    | (df_cleaned["desc_epigrafe"] == "OTROS SERVICIOS DE COMIDAS EN INSTALACIONES DEPORTIVAS, OFICINAS, EMPRESAS O SIMILARES")\
    | (df_cleaned["desc_epigrafe"] == "SERVICIOS DE COMEDOR EN CENTROS DE ACTIVIDADES DE SERVICIOS SOCIALES")\
    | (df_cleaned["desc_epigrafe"] == "SALONES DE BANQUETES Y PROVISION COMIDAS PARA EVENTOS")\
    | (df_cleaned["desc_epigrafe"] == "SERVICIOS DE PREPARACIÓN DE COMIDAS EN HOSPITALES")\
    | (df_cleaned["desc_epigrafe"] == "CIBER-CAFE")\


df_cleaned = df_cleaned[~mask]


In [13]:
df_cleaned= df_cleaned.drop(["index","fx_carga","id_epigrafe","id_clase_ndp_edificio",\
    "desc_seccion_censal_local","id_terraza","Fecha_Dec_Lic","cod_barrio_local_x",\
    "coordenada_x_local_y","coordenada_y_local_y","id_situacion_local_y",\
    "desc_situacion_local_y","hora_ini_LJ_es","hora_fin_LJ_es","desc_periodo_terraza",\
    "id_periodo_terraza","desc_tipo_licencia","desc_ubicacion_terraza","id_planta_agrupado",\
    'ref_licencia','id_tipo_licencia','id_tipo_situacion_licencia','desc_tipo_situacion_licencia',\
    "id_seccion",'desc_seccion','id_division','id_situacion_terraza','sillas_es','desc_division',\
    'cod_barrio_local_y','id_barrio_local','id_distrito_local','id_situacion_local_x'], axis=1)

- La informacion con respecto a las terrazas permanece sin conocerse, rellenar esta informacion con "sin informacion"

In [15]:
visualization_of_nan(df_cleaned)

Column 'desc_situacion_terraza' has 13197 NaN values.
Column 'Superficie_ES' has 13197 NaN values.
Column 'mesas_es' has 13197 NaN values.


In [16]:
df_cleaned[['desc_situacion_terraza','Superficie_ES','mesas_es']]= 'sin informacion'

In [17]:
visualization_of_nan(df_cleaned)

In [21]:
mask=(df_cleaned[df_cleaned["coordenada_x_local_x"]==0])


In [23]:
total= cleaner(df_cleaned)

In [24]:


total[['latitud', 'longitud']] = total.apply(lambda row: hourglass(row['coordenada_x_local_x'],\
                                    row['coordenada_y_local_x']), axis=1)



In [25]:
total = total.reindex(columns=[
'id_local',\
'desc_situacion_local_x',\
'desc_epigrafe',\
'rotulo',\
'desc_distrito_local',\
'desc_barrio_local',\
'clase_vial_acceso',\
'desc_vial_acceso',\
'coordenada_x_local_x',\
'coordenada_y_local_x',\
'desc_situacion_terraza',\
'Superficie_ES',\
'mesas_es',\
'latitud',\
'longitud']).copy()

In [26]:
total= total.drop(['coordenada_x_local_x','coordenada_y_local_x'],axis=1)

- Codigo para ubicar restaurantes, al hacer una carga de trabajo demasiado grande es mas conveniente trabajar con un heatmap, de esta manera no se peta la memoria del equipo

In [29]:
# import folium
# map_madrid = folium.Map(location=[40.4168, -3.7038], zoom_start=14)

# for (index, row) in total.iterrows():
#     folium.Marker(location = [row.loc["latitud"], row.loc["longitud"]],
#     popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
#     tooltip = "click").add_to(map_madrid)

# map_madrid


- Heatmap

In [31]:
import folium
def generateBaseMap(loc, zoom=11, tiles='cartodb positron',width=100, height=100):
    return folium.Map(location=loc, 
                      control_scale=True, 
                      zoom_start=zoom,
                      tiles=tiles)
  

MAP = [40.426558, -3.700788] 
  
base_map = generateBaseMap(MAP)


map_values = total[['latitud','longitud',"dummies"]]
map_values['weight']=map_values['dummies']


map_values1 = map_values[['latitud','longitud','dummies']]

data = map_values1.values.tolist()
           
hm = folium.plugins.HeatMap(data,gradient={0.5: 'blue', 0.2: 'lime', 0.5: 'yellow', 0.7: 'orange', 1: 'red'}, 
                opacity=8, 
                radius=11,
                use_local_extrema=False)
base_map.add_child(hm)

/var/folders/6n/5wb0sk457wnfs24brxzdhk3r0000gn/T/ipykernel_18814/990081182.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_values['weight']=map_values['dummies']


In [32]:
def features_ing(total:pd.DataFrame)->pd.DataFrame:
    np.random.seed(42)
    cuisine= ['Española','India','China','Japonesa','Italiana','Latina']
    total['tipo_de_cocina']= np.random.choice(cuisine,size=len(total))
    total['delivery']= np.random.choice(['si','no'], size= len(total))
    total['reservable']= np.random.choice(['si','no'], size= len(total))
    total['desayuno']= np.random.choice(['si','no'], size= len(total))
    total['brunch']= np.random.choice(['si','no'], size= len(total))
    total['almuerzo']= np.random.choice(['si','no'], size= len(total))
    total['cena']= np.random.choice(['si','no'], size= len(total))
    total['para_llevar']= np.random.choice(['si','no'], size= len(total))
    total['comida_vegetariana']= np.random.choice(['si','no'], size= len(total))
    total['rango_de_precio']= np.random.randint(0,5,size=len(total))
    total['rating']= np.random.uniform(0,5,size=len(total)).round(1)
    return total


In [33]:
total= features_ing(total)

In [34]:
df= total.rename(columns={'id_local':'key','desc_situacion_local_x':'abierto_o_cerrado',\
                        'desc_epigrafe':'tipo_de_establecimiento','rotulo':'nombre',\
                        'desc_distrito_local':'distrito','clase_vial_acceso':'acceso','desc_vial_acceso':'nombre_calle',\
                        'desc_situacion_terraza':'terraza_abierta_o_cerrada','Superficie_ES':'terraza_en_metros',\
                        'mesas_es':'cantidad_mesas_terraza'})

In [39]:
df=df.drop_duplicates('key')


In [40]:
df.to_csv('../notebooks/data/cleaned20242108.csv')

In [43]:
cleaned= pd.read_csv('Users/main/desktop/google_api/data/cleaned20242108.csv',index_col=0,skipinitialspace=True)

In [44]:
cleaned.columns= cleaned.columns.str.strip()
connection= sqlite3.connect('/Users/main/Desktop/llm.db')
cleaned.to_sql('restaurants_madrid',connection,if_exists='replace')
connection.close()

In [10]:
cleaned= pd.read_csv('/Users/main/Desktop/google_api/data/cleaned20242108.csv',index_col=0)

In [24]:
chamartin=cleaned[cleaned['distrito']=='CHAMARTIN           ']

In [26]:
chamartin.head(2)

,key,abierto_o_cerrado,tipo_de_establecimiento,nombre,distrito,desc_barrio_local,acceso,nombre_calle,terraza_abierta_o_cerrada,terraza_en_metros,cantidad_mesas_terraza,latitud,longitud,tipo_de_cocina,delivery,reservable,desayuno,brunch,almuerzo,cena,para_llevar,comida_vegetariana,rango_de_precio,rating
1033,50000178,Abierto,RESTAURANTE,LA TARANTELLA,CHAMARTIN,HISPANOAMERICA,CALLE,PADRE DAMIAN,sin informacion,sin informacion,sin informacion,40.457966,-3.685381,Italiana,no,si,no,si,no,si,no,si,2,4.2
1034,50000184,Abierto,BAR RESTAURANTE,LA TONA,CHAMARTIN,PROSPERIDAD,CALLE,CLARA DEL REY,sin informacion,sin informacion,sin informacion,40.446023,-3.667937,Japonesa,si,si,si,si,si,no,no,si,0,0.4
